In [ ]:
import json
from collections import defaultdict

# Paths to the trace files (from multiple services)
trace_files = [
    "./executed/202503261213-hotel-test1/traces_frontend.json",
    "./executed/202503261213-hotel-test1/traces_geo.json",
    "./executed/202503261213-hotel-test1/traces_profile.json",
    "./executed/202503261213-hotel-test1/traces_rate.json",
    "./executed/202503261213-hotel-test1/traces_recommendation.json",
    "./executed/202503261213-hotel-test1/traces_reservation.json",
    "./executed/202503261213-hotel-test1/traces_search.json",
]

# traceID -> list of spans
trace_spans = defaultdict(list)

# traceID -> processID -> serviceName
trace_services = defaultdict(dict)

trace_spans.clear()
trace_services.clear()

seen_span_ids = defaultdict(set)

# Load and merge traces from all files
for file_path in trace_files:
    with open(file_path, 'r') as f:
        data = json.load(f)
        for trace in data.get("data", []):
            trace_id = trace.get("traceID")

            # Store processID -> serviceName mapping
            processes = trace.get("processes", {})
            for pid, process in processes.items():
                service_name = process.get("serviceName", "<unknown>")
                trace_services[trace_id][pid] = service_name

            # Collect all spans
            for span in trace.get("spans", []):
                span_id = span.get("spanID")
                if span_id not in seen_span_ids[trace_id]:
                    seen_span_ids[trace_id].add(span_id)
                    trace_spans[trace_id].append(span)

# Print traces ordered by span startTime, including service name and span ID
for trace_id, spans in trace_spans.items():
    print(f"Trace ID: {trace_id}")
    sorted_spans = sorted(spans, key=lambda s: s.get("startTime", 0))
    for i, span in enumerate(sorted_spans, start=1):
        op_name = span.get("operationName", "<unknown>")
        process_id = span.get("processID", "")
        span_id = span.get("spanID", "")
        service_name = trace_services[trace_id].get(process_id, "<unknown service>")
        print(f"  Step {i}: [{service_name}] {op_name} (spanID: {span_id})")
    print()


FileNotFoundError: [Errno 2] No such file or directory: './executed/202503252224-hotel-test1/traces_frontend.json'